In [56]:
pip install pandas scipy


Note: you may need to restart the kernel to use updated packages.


In [84]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.device_count())  # Should be >= 1
print(torch.cuda.get_device_name(0))  # Name of GPU


True
1
NVIDIA GeForce RTX 4060 Laptop GPU


In [85]:
import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0)




(True, 'NVIDIA GeForce RTX 4060 Laptop GPU')

In [86]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
import re
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split




In [99]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder, StandardScaler
import os

def interval_to_mid(x):
    """Convert interval strings to midpoints, e.g., '20-30' => 25.0"""
    if pd.isna(x): return x
    parts = re.findall(r'-?[\d\.]+|inf', str(x))
    if len(parts) == 2:
        lo, hi = parts
        lo = float(lo) if lo not in ("-inf", "inf") else float(hi)
        hi = float(hi) if hi not in ("-inf", "inf") else float(lo)
        return (lo + hi) / 2
    try:
        return float(x)
    except:
        return pd.NA

def preprocess_dataset(file_path, target_column, interval_cols=None, output_path="processed_data.csv"):
    # Load dataset
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()


    # Auto-detect interval columns if not provided
    if interval_cols is None:
        interval_cols = df.select_dtypes(include=['object']).columns[
            df.select_dtypes(include=['object']).apply(
                lambda col: col.astype(str).str.contains(r'\d+\s*-\s*\d+').any()
            )
        ].tolist()

    # Convert intervals to midpoints
    for col in interval_cols:
        df[col] = df[col].apply(interval_to_mid)

    # Separate types
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    if target_column in numeric_cols:
        numeric_cols.remove(target_column)

    # Encode target
    le = LabelEncoder()
    df[target_column] = le.fit_transform(df[target_column])

    # Scale numeric columns if they exist
    if numeric_cols:
        scaler = StandardScaler()
        df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
        df_numeric = df[numeric_cols].reset_index(drop=True)
    else:
        df_numeric = pd.DataFrame()

    # Identify categorical columns
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

    # One-hot encode categorical columns if they exist
    if cat_cols:
        df_cat = pd.get_dummies(df[cat_cols], drop_first=False).astype(int).reset_index(drop=True)
    else:
        df_cat = pd.DataFrame()

    # Target column
    df_target = df[[target_column]].reset_index(drop=True)

    # Final dataset
    df_processed = pd.concat([df_numeric, df_cat, df_target], axis=1)

    # Hyperparameter adjustment
    num_rows = df_processed.shape[0]
    epochs = 100 if num_rows < 50000 else 150
    print(f"📊 Dataset size: {num_rows} rows → EPOCHS = {epochs}")

    return df_processed, numeric_cols, cat_cols, epochs, target_column,numeric_cols


In [100]:
# Example usage:
data_path = "C:/Users/pasin/Downloads/credit-a_converted.csv"
target = "class"
interval_columns = []
output_file = "processed_data.csv"
#["age", "education-num", "capital-gain", "capital-loss", "hours-per-week"]

# TARGET_COL        = "class"
# TARGET_COL        = "class"
df_processed, numeric_cols, cat_cols, EPOCHS,TARGET_COL,NUMERIC_COLS  = preprocess_dataset(
    file_path=data_path,
    target_column=target,
    interval_cols=interval_columns,
    output_path=output_file
)


📊 Dataset size: 690 rows → EPOCHS = 100


In [101]:
pip install torch torchvision scipy scikit-learn xgboost --quiet


Note: you may need to restart the kernel to use updated packages.


In [102]:
# Installs & Imports

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.linear_model    import LogisticRegression
from sklearn.neural_network  import MLPClassifier
from sklearn.ensemble        import RandomForestClassifier
from xgboost                 import XGBClassifier
from sklearn.model_selection import KFold
from scipy.stats             import wasserstein_distance
from scipy.spatial.distance  import jensenshannon

# Hyperparameters
PREPROCESSED_PATH = "C:/Users/pasin/Downloads/processed_data.csv"
LATENT_DIM        = 100
BATCH_SIZE        = 64
# EPOCHS            = 100    # medium dataset
REPEATS           = 3
FOLDS             = 2
SYN_RATIO         = 0.5    # synthetic / real train size
# NUMERIC_COLS      = ["age","education-num","capital-gain","capital-loss","hours-per-week"]
# TARGET_COL        = "class"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# CR-GAN Definitions
class Generator(nn.Module):
    def __init__(self, z_dim, out_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim,256), nn.ReLU(),
            nn.Linear(256,512),   nn.ReLU(),
            nn.Linear(512,256),   nn.ReLU(),
            nn.Linear(256,out_dim),
            nn.Tanh()
        )
    def forward(self, z):
        return self.net(z)

class Discriminator(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim,512), nn.ReLU(),
            nn.Linear(512,256),   nn.ReLU(),
            nn.Linear(256,128),   nn.ReLU(),
            nn.Linear(128,1),     nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

def train_cramer_gan(G, D, loader, epochs):
    G, D = G.to(device), D.to(device)
    optg = optim.Adam(G.parameters(), lr=2e-4)
    optd = optim.Adam(D.parameters(), lr=2e-4)
    loss_fn = nn.BCELoss()
    for ep in range(1, epochs+1):
        for real_batch, _ in loader:
            real_batch = real_batch.to(device)
            bsz = real_batch.size(0)
            # — D step
            optd.zero_grad()
            z      = torch.randn(bsz, LATENT_DIM, device=device)
            fake   = G(z).detach()
            d_real = D(real_batch)
            d_fake = D(fake)
            lossd  = loss_fn(d_real, torch.ones_like(d_real)) + \
                     loss_fn(d_fake, torch.zeros_like(d_fake))
            lossd.backward();  optd.step()
            # — G step
            optg.zero_grad()
            z     = torch.randn(bsz, LATENT_DIM, device=device)
            fake2 = G(z)
            dg    = D(fake2)
            lossg = loss_fn(dg, torch.ones_like(dg))
            lossg.backward(); optg.step()
        if ep%20==0 or ep==1 or ep==epochs:
            print(f" Ep {ep}/{epochs}  D_loss={lossd.item():.4f}  G_loss={lossg.item():.4f}")
    return G, D

def generate_synthetic(G, n_samples):
    G = G.to(device).eval()
    with torch.no_grad():
        z    = torch.randn(n_samples, LATENT_DIM, device=device)
        data = G(z).cpu().numpy()
    return data

def compute_tstr_all(X_real, y_real, X_syn, y_syn):
    # Train each classifier on synthetic → score on real
    results = {}
    for name, clf in [
        ("LR",  LogisticRegression(max_iter=5000)),
        ("MLP", MLPClassifier(hidden_layer_sizes=(128,64), max_iter=1000)),
        ("RF",  RandomForestClassifier(n_estimators=200)),
        ("XGB", XGBClassifier(eval_metric="logloss"))
    ]:
        clf.fit(X_syn, y_syn)
        results[name] = clf.score(X_real, y_real)*100.0
    return results

def compute_jsd_wd(X_real, X_syn, num_idx):
    jsd_list, wd_list = [], []
    for i in num_idx:
        p_real, _  = np.histogram(X_real[:,i], bins=50, density=True)
        p_syn,  _  = np.histogram(X_syn[:,i], bins=50, density=True)
        jsd_list.append( jensenshannon(p_real, p_syn) )
        wd_list .append( wasserstein_distance(X_real[:,i], X_syn[:,i]) )
    return np.mean(jsd_list), np.mean(wd_list)

# Load Preprocessed Data
df = pd.read_csv(PREPROCESSED_PATH)
X_full = df.drop(columns=[TARGET_COL]).values.astype(np.float32)
y_full = df[TARGET_COL].values.astype(int)

if NUMERIC_COLS:
    num_idx = [df.columns.get_loc(c) for c in NUMERIC_COLS if c in df.columns]
else:
    num_idx = []


# 3×(2-Fold CV)
tstr_scores = {m:[] for m in ["LR","MLP","RF","XGB"]}
jsd_scores, wd_scores = [], []

kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
for rep in range(1, REPEATS+1):
    for fold,(train_idx, test_idx) in enumerate(kf.split(X_full),1):
        print(f"\n— Rep {rep}/{REPEATS}  Fold {fold}/{FOLDS} —")
        X_tr, X_te = X_full[train_idx], X_full[test_idx]
        y_tr, y_te = y_full[train_idx], y_full[test_idx]
        loader = DataLoader(
            TensorDataset(torch.from_numpy(X_tr), torch.from_numpy(y_tr)),
            batch_size=BATCH_SIZE, shuffle=True
        )
        # train
        G = Generator(LATENT_DIM, X_tr.shape[1])
        D = Discriminator(X_tr.shape[1])
        G, D = train_cramer_gan(G, D, loader, epochs=EPOCHS)
        # synth
        n_syn = int(SYN_RATIO * len(X_tr))
        X_syn = generate_synthetic(G, n_syn)
        y_syn = np.random.choice(y_tr, size=n_syn, replace=True)
        # metrics
        tstrs = compute_tstr_all(X_te, y_te, X_syn, y_syn)
        for m,sc in tstrs.items(): tstr_scores[m].append(sc)
        js, wd = compute_jsd_wd(X_te, X_syn, num_idx)
        jsd_scores.append(js);  wd_scores.append(wd)

# Report CV Results
print("\n=== CV Results (mean ± std) ===")
for m in ["LR","MLP","RF","XGB"]:
    arr = np.array(tstr_scores[m])
    print(f" • {m:4s} TSTR = {arr.mean():.2f}% ± {arr.std():.2f}%")
print(f" • JSD = {np.mean(jsd_scores):.4f} ± {np.std(jsd_scores):.4f}")
print(f" • WD  = {np.mean(wd_scores):.4f} ± {np.std(wd_scores):.4f}")

# Train on Full & Save Final Synthetic
# retrain on all data
full_loader = DataLoader(
    TensorDataset(torch.from_numpy(X_full), torch.from_numpy(y_full)),
    batch_size=BATCH_SIZE, shuffle=True
)
Gf = Generator(LATENT_DIM, X_full.shape[1])
Df = Discriminator(X_full.shape[1])
Gf, Df = train_cramer_gan(Gf, Df, full_loader, epochs=EPOCHS)

# generate 50% synthetic
n_final = int(SYN_RATIO * len(X_full))
Xf_syn = generate_synthetic(Gf, n_final)
yf_syn = np.random.choice(y_full, size=n_final, replace=True)


cols = df.columns[:-1]
syn_df = pd.DataFrame(Xf_syn, columns=cols)
syn_df[TARGET_COL] = yf_syn
out_path = "/content/drive/MyDrive/Katabatic/Data/Adult/synthetic4_adult_final.csv"
syn_df.to_csv(out_path, index=False)
print(f"\nSaved final synthetic dataset ({n_final} rows) to:\n  {out_path}")


 Ep 20/100  D_loss=0.2782  G_loss=3.8647
 Ep 40/100  D_loss=0.1103  G_loss=4.3077


KeyboardInterrupt: 

In [11]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     --------------------------- ------------ 2.9/4.1 MB 13.9 MB/s eta 0:00:01
     ---------------------------------------- 4.1/4.1 MB 11.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     --------------------------- ------------ 2.9/4.1 MB 14.0 MB/s eta 0:00:01
     ---------------------------------------- 4.1/4.1 MB 13.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     ---------------------------- ----------- 2.9/4.1 MB 13.9 MB/s eta 0:00:01
     ---------------------------------------- 4.1/4.1 MB 13.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     ------------------------------ 

  You can safely remove it manually.
